In [1]:
import MeCab
import pickle
import gensim
import numpy as np
import pandas as pd
import gensim.corpora as corpora
from collections import Counter
from gensim import similarities
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.fasttext import FastText
from pprint import pprint
from google.cloud import storage as gcs
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, fbeta_score, classification_report, confusion_matrix
from sklearn.model_selection import KFold, train_test_split, RandomizedSearchCV, GridSearchCV, ShuffleSplit, cross_val_score
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE

//anaconda3/envs/jctensorflow/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
//anaconda3/envs/jctensorflow/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
//anaconda3/envs/jctensorflow/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  depr

In [2]:
INPUT_BUCKET = 'gs://jesseulundo-bucket/questionnaire_classification/input_data/'
PROJECT_NAME = 'linkbal-dp'
BUCKET_NAME = 'mj-labeling-questionnaires-dev.datasets.linkbal.com' 
client = gcs.Client(PROJECT_NAME)
bucket = gcs.Bucket(client, name=BUCKET_NAME)
# JAPANESE STOPWORDS
jp_stop_words = pd.read_csv('gs://jesseulundo-bucket/questionnaire_classification/input_data/Japanese_stopword_list.txt', header=None, delimiter="\t")
# HAND LABELLED QUESTIONNAIRE DATA
qs = pd.read_csv(INPUT_BUCKET + 'new_test_data_20200310.csv', sep=',')
qs 

//anaconda3/envs/jctensorflow/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
//anaconda3/envs/jctensorflow/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.go

,id,overall_satisfaction,overall_satisfaction_reason,categories
0,111377,2,店舗スタッフの対応が悪い,スタッフの対応が悪い
1,111379,2,他のボドゲのお店だと店員や常連さんが気をきかせてくれていたので、そこと比べてしまうとどうして...,スタッフの対応が悪い
2,111381,2,想像していた人数よりも少ない。,参加人数（多い、少ない）
3,111383,2,大人数なのにマンツーマン、フリータイムが少ない。,出会えなかった
4,111386,2,食べ物が酒の肴にならない,飲食内容の質が低い
...,...,...,...,...
4404,131663,1,参加者が少なすぎる,参加人数（多い、少ない）
4405,131664,1,料理の品数が少なすぎた。\nスタッフの対応も最悪。,飲食内容の質が低い
4406,131675,1,最終的に社長から返金対応する旨の提案を受け、案内を約束されたが未だ待っている状況。\n本当に...,その他
4407,131676,2,水上バスは夜景がきれいだったけど、ほんとにそれだけって感じだった。違和感がありすぎた。,会場や雰囲気が悪い


In [3]:
qs = qs.dropna()
#qs = qs.replace(to_replace ="男女比があっていない\n", value ="男女比があっていない")

In [4]:
qs['categories'].unique()

array(['スタッフの対応が悪い', '参加人数（多い、少ない）', '出会えなかった', '飲食内容の質が低い', 'その他',
       '会場や雰囲気が悪い', '参加条件が合ってない', '男女比があっていない', '掲載内容と相違がある',
       '男女比があっていない\n'], dtype=object)

In [5]:
category_codes = {'スタッフの対応が悪い': 0,
                  '参加人数（多い、少ない)': 1,
                  '出会えなかった': 2,
                  '飲食内容のが質が低い': 3,
                  '会場や雰囲気が悪い': 4,
                  '参加条件が合っていない': 5,
                  '男女比があっていない': 6,
                  '掲載内容と相違がある': 7,
                  'その他': 8}